**# Import Library**

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from tensorflow.keras import regularizers
import time

**# Open Main Directory**

In [ ]:
cd /content/drive/MyDrive/Colab Notebooks/TIMESERIES

/content/drive/MyDrive/Colab Notebooks/TIMESERIES


**# Import Dataset**

In [ ]:
data = pd.read_csv('household_power_consumption.txt', sep=';', parse_dates={'dt' : ['Date', 'Time']}, infer_datetime_format=True, 
                 low_memory=False, na_values=['nan','?'], index_col='dt')

data.head()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
dt,,,,,,,
2006-12-16 17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0
2006-12-16 17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0
2006-12-16 17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0
2006-12-16 17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0
2006-12-16 17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0


**# Shape of Dataset**

In [ ]:
data.shape

(2075259, 7)

**# Information of Dataset**

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2075259 entries, 2006-12-16 17:24:00 to 2010-11-26 21:02:00
Data columns (total 7 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Global_active_power    float64
 1   Global_reactive_power  float64
 2   Voltage                float64
 3   Global_intensity       float64
 4   Sub_metering_1         float64
 5   Sub_metering_2         float64
 6   Sub_metering_3         float64
dtypes: float64(7)
memory usage: 126.7 MB


**# Description of Dataset**

In [ ]:
data.describe()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
count,2.049280e+06,2.049280e+06,2.049280e+06,2.049280e+06,2.049280e+06,2.049280e+06,2.049280e+06
mean,1.091615e+00,1.237145e-01,2.408399e+02,4.627759e+00,1.121923e+00,1.298520e+00,6.458447e+00
std,1.057294e+00,1.127220e-01,3.239987e+00,4.444396e+00,6.153031e+00,5.822026e+00,8.437154e+00
min,7.600000e-02,0.000000e+00,2.232000e+02,2.000000e-01,0.000000e+00,0.000000e+00,0.000000e+00
25%,3.080000e-01,4.800000e-02,2.389900e+02,1.400000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,6.020000e-01,1.000000e-01,2.410100e+02,2.600000e+00,0.000000e+00,0.000000e+00,1.000000e+00
75%,1.528000e+00,1.940000e-01,2.428900e+02,6.400000e+00,0.000000e+00,1.000000e+00,1.700000e+01
max,1.112200e+01,1.390000e+00,2.541500e+02,4.840000e+01,8.800000e+01,8.000000e+01,3.100000e+01


**# Check NaN Values**

In [ ]:
data.isnull().sum()

Global_active_power      25979
Global_reactive_power    25979
Voltage                  25979
Global_intensity         25979
Sub_metering_1           25979
Sub_metering_2           25979
Sub_metering_3           25979
dtype: int64

**# Fill NaN Values with Mean Value**

In [ ]:
data = data.fillna(data.mean())
data.isnull().sum()

Global_active_power      0
Global_reactive_power    0
Voltage                  0
Global_intensity         0
Sub_metering_1           0
Sub_metering_2           0
Sub_metering_3           0
dtype: int64

**# Convert TimeSeries to Supervised Learning**

In [ ]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    dff = pd.DataFrame(data)
    cols, names = list(), list()
    for i in range(n_in, 0, -1):
        cols.append(dff.shift(-i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    for i in range(0, n_out):
        cols.append(dff.shift(-i))
        if i==0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1)) for j in range(n_vars)]        
        agg = pd.concat(cols, axis=1)
        agg.columns = names
        if dropnan:
            agg.dropna(inplace=True)
        return agg

**# Normalize Dataset**

In [ ]:
from sklearn.preprocessing import MinMaxScaler

norm = data.values
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(norm)
reframed = series_to_supervised(scaled, 1, 1)
r = list(range(data.shape[1]+1, 2*data.shape[1]))
reframed.drop(reframed.columns[r], axis=1, inplace=True)
reframed.head()

,var1(t-1),var2(t-1),var3(t-1),var4(t-1),var5(t-1),var6(t-1),var7(t-1),var1(t)
0,0.478363,0.313669,0.336995,0.473029,0.0,0.0125,0.516129,0.374796
1,0.479631,0.358273,0.326010,0.473029,0.0,0.0250,0.548387,0.478363
2,0.480898,0.361151,0.340549,0.473029,0.0,0.0125,0.548387,0.479631
3,0.325005,0.379856,0.403231,0.323651,0.0,0.0125,0.548387,0.480898
4,0.311787,0.375540,0.381906,0.307054,0.0,0.0250,0.548387,0.325005


**# Split Dataset into 80 % Train & 20% Val Dataset**

In [ ]:
values = reframed.values
n_train_time = int(len(values) * 0.8)
train = values[:n_train_time, :]
test = values[n_train_time:, :]
train_x, train_y = train[:, :-1], train[:, -1]
test_x, test_y = test[:, :-1], test[:, -1]
train_x = train_x.reshape((train_x.shape[0], 1, train_x.shape[1]))
test_x = test_x.reshape((test_x.shape[0], 1, test_x.shape[1]))

print('Total Dataset: %d' % (len(values)))
print('Training Dataset: %d' % (len(train)))
print('Validation Dataset: %d' % (len(test)))

Total Dataset: 2075258
Training Dataset: 1660206
Validation Dataset: 415052


**# Create TimeSeries Model**

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(128, input_shape=(train_x.shape[1], train_x.shape[2]), return_sequences = True, dropout=0.25),
    tf.keras.layers.LSTM(64, dropout=0.1),
    tf.keras.layers.Dense(512, activation='relu',kernel_regularizer=regularizers.l2(0.001)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(128, activation='relu',kernel_regularizer=regularizers.l2(0.001)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(64, activation='relu',kernel_regularizer=regularizers.l2(0.001)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(1)
    ])

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 1, 128)            69632     
_________________________________________________________________
lstm_5 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense_8 (Dense)              (None, 512)               33280     
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 128)               65664     
_________________________________________________________________
dropout_7 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 64)               

**# Define Loss, Optimizer & Compile the Model**

In [ ]:
model.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer=tf.keras.optimizers.Adam(learning_rate=2e-04), metrics=['mae'])

**# Create Callback**

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('mae')>0.1):
      self.model.stop_training = True
callbacks = myCallback()

**# Training the Model**

In [ ]:
start = time.time()

history = model.fit(train_x, train_y, epochs=10, 
                    validation_data=(test_x, test_y),
                    callbacks = myCallback(),
                    verbose=1,
                    shuffle=False)

end = time.time()

elapsed_time = end - start
time.strftime("Time Elapsed: %H:%M:%S", time.gmtime(elapsed_time))

Epoch 1/10
51882/51882 [==============================] - 403s 8ms/step - loss: 0.0037 - mae: 0.0263 - val_loss: 0.0015 - val_mae: 0.0234
Epoch 2/10
51882/51882 [==============================] - 389s 7ms/step - loss: 0.0016 - mae: 0.0214 - val_loss: 0.0013 - val_mae: 0.0234
Epoch 3/10
51882/51882 [==============================] - 394s 8ms/step - loss: 0.0016 - mae: 0.0210 - val_loss: 0.0014 - val_mae: 0.0229
Epoch 4/10
51882/51882 [==============================] - 396s 8ms/step - loss: 0.0015 - mae: 0.0208 - val_loss: 0.0014 - val_mae: 0.0242
Epoch 5/10
51882/51882 [==============================] - 405s 8ms/step - loss: 0.0015 - mae: 0.0208 - val_loss: 0.0014 - val_mae: 0.0236
Epoch 6/10
51882/51882 [==============================] - 406s 8ms/step - loss: 0.0015 - mae: 0.0207 - val_loss: 0.0017 - val_mae: 0.0251
Epoch 7/10
51882/51882 [==============================] - 407s 8ms/step - loss: 0.0015 - mae: 0.0207 - val_loss: 0.0013 - val_mae: 0.0238
Epoch 8/10
51882/51882 [==========

'Time Elapsed: 01:07:25'